In [1]:
import os
import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe
from pandas import ExcelWriter

In [2]:
#code from Agata
#these are nice functions to open LIMS, make a query and then close LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        
        #THESE ARE IMPORTANT!!!!!!
        #Every query needs to be closed when done
        cursor.close()             
        conn.close()
    return results


#this last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

# Make user report

In [3]:
def user_df(P_number):
    """Saves an excel dataframe with all patched cells for a specified user in current directory.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
       
    Returns
    -------
    None
    """
    user = master_df[master_df["patched_cell_container"].str.contains(P_number)]
    user.sort_values("patched_cell_container", axis = 0, inplace = True)
    user = user.loc[:,['name', 'patched_cell_container','date', 'rna_qc', 'go_no_go_63x', 'link_20x']]
    writer = pd.ExcelWriter('user_dataframe.xlsx')
    user.to_excel(writer,'Sheet1')
    return None


In [4]:
def cell_count(P_number, df):
    """Return the number of cells in a specified dataframe for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
    df: a pandas dataframe
    
    Returns
    -------
    An integer cell number
    """
    user = df[df["patched_cell_container"].str.contains(P_number)]
    return user["patched_cell_container"].count()
    

In [5]:
def gen_filter(df, col, cond):
    filtered = df[df[col] == cond]
    return filtered

In [6]:
def per_user(P_number):
    """Returns a summary of patcher metrics for a specified user.
     
    Parameters
    ----------
    P_number : A string. Corresponds to the user's P number in the form of P1, P2, P3, etc.
        
    Returns
    -------
    Returns None
    Prints a summary of patcher metrics, including total cells patched, cells that passed RNA seq QC, and QC pass percentage
    """
    print "Total cells patched:", cell_count(P_number,master_df)
    print "Cells that passed RNA seq QC:", cell_count(P_number,qcpass)
    print "QC pass percentage:", float((cell_count(P_number,qcpass)))/float((cell_count(P_number,master_df)))
    print "Cells imaged at 63x:", cell_count(P_number,imagego)
    
    

In [7]:
query = """
WITH do_63x(cell_id, go) AS (SELECT DISTINCT cell.id, array_to_string(array_agg(DISTINCT tag.name), ' _AND_ ') 
FROM specimens cell JOIN ephys_roi_results err ON err.id = cell.ephys_roi_result_id 
JOIN specimen_tags_specimens sptagsp ON sptagsp.specimen_id = cell.id 
JOIN specimen_tags tag ON tag.id = sptagsp.specimen_tag_id AND tag.id in (602120185,602122082) 
GROUP BY cell.id ORDER BY 1), 
imgs20 AS (WITH tag_list AS (SELECT DISTINCT ims.id AS ims_id, imst.name AS tag FROM image_series ims 
JOIN image_series_image_series_tags ims2imst ON ims2imst.image_series_id = ims.id 
JOIN image_series_tags imst ON imst.id = ims2imst.image_series_tag_id 
ORDER BY 1,2) 
SELECT DISTINCT ims.id AS image_series_id, ims.workflow_state AS workflow_state, ims.specimen_id, 
ARRAY_TO_STRING(ARRAY_AGG(DISTINCT tag_list.tag), '_AND_') AS tags FROM image_series ims 
LEFT JOIN tag_list ON tag_list.ims_id = ims.id 
WHERE ims.type = 'FocalPlaneImageSeries' AND is_stack = false 
GROUP BY ims.id, ims.workflow_state, ims.specimen_id)
SELECT DISTINCT s.patched_cell_container, s.created_at AS date, do_63x.go AS go_no_go_63x, CASE WHEN ra.id IS NOT NULL THEN (CASE WHEN ra.failed = 't' THEN 'failed' ELSE 'passed' END) ELSE '' END AS rna_qc, s.name,
'=HYPERLINK("http://lims2/focal_plane_image_series?id=' || imgs20.image_series_id || '")' as link_20x
FROM specimens s 
LEFT JOIN projects proj ON s.project_id = proj.id 
LEFT JOIN do_63x ON do_63x.cell_id = s.id 
JOIN specimens slice ON s.parent_id = slice.id
LEFT JOIN rna_amplification_inputs rai on rai.sample_id = s.id 
LEFT JOIN imgs20 ON slice.id = imgs20.specimen_id 
LEFT JOIN rna_amplifications ra on ra.id = rai.rna_amplification_id 
WHERE proj.code <> 'mMPATCH' AND 
s.patched_cell_container IS NOT NULL 
AND s.created_at > '2017-09-30'
"""

#LEFT JOIN ephys_roi_results err ON s.ephys_roi_result_id = err.id
#AND (err.recording_date > '2017-10-01' OR err.recording_date IS NULL)
#AND err.recording_date > '2017-10-01'

master_df = get_lims_dataframe(query)
master_df

,go_no_go_63x,name,rna_qc,patched_cell_container,link_20x,date
0,None,Ntsr1-Cre_GN220;Ai14-361815.05.02.01,failed,PAS4_171215_451_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-15 18:40:32.255491
1,None,H18.03.010.11.14.11,,P8S4_180508_367_A01,None,2018-05-09 06:45:39.002788
2,None,Nkx2-1-CreERT2;Ai14-357183.04.02.01,failed,P8S4_171110_351_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-11-10 18:30:27.594198
3,63x no go,Rbp4-Cre_KL100;Ai14-374885.07.02.02,failed,P8S4_180216_352_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-16 19:00:30.895698
4,None,Tlx3-Cre_PL56;Ai14-375777.07.01.02,failed,P9S4_180223_406_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-02-23 22:25:32.450008
5,None,Pvalb-IRES-Cre;Ai14-388062.02.02.03,passed,PBS4_180501_510_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-05-02 01:50:39.101353
6,63x no go,Vip-IRES-Cre;Ai14-354376.04.02.01,passed,P8S4_171009_357_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-10-09 22:50:41.079759
7,63x go,Crh-IRES-Cre_ZJH;Sst-IRES-FlpO;Ai65-362355.04....,passed,P8S4_171129_353_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-11-29 19:10:31.796143
8,None,Rbp4-Cre_KL100;Ai14-383392.04.01.02,passed,PBS4_180410_504_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2018-04-10 22:00:44.037086
9,63x go,Slc32a1-IRES2-FlpO;Ai65F-363946.05.02.01,passed,PAS4_171219_451_A01,"=HYPERLINK(""http://lims2/focal_plane_image_ser...",2017-12-19 18:35:31.986715


In [8]:
qcpass = gen_filter(master_df,'rna_qc', 'passed')
#qcpass

In [9]:
imagego = gen_filter(master_df,'go_no_go_63x', '63x go')

In [10]:
per_user('P8')

Total cells patched: 773
Cells that passed RNA seq QC: 539
QC pass percentage: 0.697283311772
Cells imaged at 63x: 185


In [11]:
user_df('P8')

C:\Users\lindsayn\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
